In [58]:
library(tidyverse)
library(spData)
library(sf)
## New Packages
library(mapview) # new package that makes easy leaflet maps
library(foreach)
library(doParallel)
?registerDoParallel
registerDoParallel(4)
getDoParWorkers() # check registered cores

[1] 4

registerDoParallel         package:doParallel          R Documentation

_r_e_g_i_s_t_e_r_D_o_P_a_r_a_l_l_e_l

_D_e_s_c_r_i_p_t_i_o_n:

     The 'registerDoParallel' function is used to register the parallel
     backend with the 'foreach' package.

_U_s_a_g_e:

     registerDoParallel(cl, cores=NULL, ...)
     stopImplicitCluster()
     
_A_r_g_u_m_e_n_t_s:

      cl: A cluster object as returned by 'makeCluster', or the number
          of nodes to be created in the cluster. If not specified, on
          Windows a three worker cluster is created and used.

   cores: The number of cores to use for parallel execution. If not
          specified, the number of cores is set to the value of
          'options("cores")', if specified, or to one-half the number
          of cores detected by the 'parallel' package.

     ...: Package options.  Currently, only the 'nocompile' option is
          supported.  If 'nocompile' is set to 'TRUE', compiler 

In [52]:
# census API key can be obtained from: http://api.census.gov/data/key_signup.html
census_api_key("c8b944c15e4c18bb6e02e85bcf22f832edb4e577", install = TRUE)

ERROR: Error: A CENSUS_API_KEY already exists. You can overwrite it with the argument overwrite=TRUE


In [59]:
readRenviron("~/.Renviron")

In [60]:
library(tidycensus)
racevars <- c(White = "P005003", 
              Black = "P005004", 
              Asian = "P005006", 
              Hispanic = "P004003")
options(tigris_use_cache = TRUE)
erie <- get_decennial(geography = "block", variables = racevars, 
                  state = "NY", county = "Erie County", geometry = TRUE,
                  summary_var = "P001001", cache_table=T) 
buff=st_crop(erie,c(xmin=-78.9,xmax=-78.85,ymin=42.888,ymax=42.92))

Getting data from the 2010 decennial Census

Using Census Summary File 1

Warning message:
"attribute variables are assumed to be spatially constant throughout all geometries"


In [62]:
buff$variable=as.factor((buff$variable))
variable_vec=as.array(unique(buff$variable))
rm(i)
points=foreach(i=1:4,.combine=rbind) %dopar%      
  {buff%>%filter(variable==unique(buff$variable)[i])%>%st_sample(size=.$value)%>%st_as_sf()%>%mutate(variable=unique(buff$variable)[i])}

Warning message in rm(i):
"object 'i' not found"


ERROR: Error in {: task 1 failed - "could not find function "%>%""


In [57]:
mapview(points,zcol='variable',cex=1,alpha=0)

ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function 'mapView' for signature '"function"'
